In [1]:
import importlib.resources as resources
import pathlib as pl

import pandas as pd
import numpy as np
from munch import Munch
from wigner_time import connection as con
from wigner_time import timeline as tl
from wigner_time import adwin as adwin
import wigner_time

from copy import deepcopy

ModuleNotFoundError: No module named 'pandas'

In [2]:
connections = con.connection(
    ["shutter_MOT", 1, 11],
    ["shutter_repump", 1, 12],
    ["shutter_imaging", 1, 13],
    ["shutter_OP001", 1, 14],
    ["shutter_OP002", 1, 15],
    ["AOM_MOT", 1, 1],
    ["AOM_repump", 1, 2],
    ["AOM_imaging", 1, 5],
    ["AOM_OP_aux", 1, 30], # should be set to 0 always
    ["AOM_OP", 1, 31],
    ["coil_compensationX__A", 4, 7],
    ["coil_compensationY__A", 3, 2],
    ["coil_MOTlower__A", 4, 1],
    ["coil_MOTupper__A", 4, 3],
    ["coil_MOTlowerPlus__A", 4, 2],
    ["coil_MOTupperPlus__A", 4, 4],
    ["lockbox_MOT__V", 3, 8],
)

devices = pd.DataFrame(
    columns=["variable", "unit_range", "safety_range"],
    data=[
        ["coil_compensationX__A", (-3, 3), (-3, 3)],
        ["coil_compensationY__A", (-3, 3), (-3, 3)],
        ["coil_MOTlower__A", (-5, 5), (-5, 5)],
        ["coil_MOTupper__A", (-5, 5), (-5, 5)],
        ["coil_MOTlowerPlus__A", (-5, 5), (-5, 5)],
        ["coil_MOTupperPlus__A", (-5, 5), (-5, 5)],
        ["lockbox_MOT__V", (-10, 10)],
    ],
)

constants=Munch(
    safety_factor=1.1,
    factor__VpMHz=0.05,
    compensation_magnetic_field__A=-0.1,
    compensation_magnetic_field_X__A=0.25,
    compensation_magnetic_field_Y__A=1.5,

    lag_MOTshutter =2.48e-3,

    OP = Munch(
        lag_aom_on=15e-6,
        lag_shutter_on=1.5e-3,
        lag_shutter_off=1.5e-3,
        duration_shutter_on=140e-6,
        duration_shutter_off=600e-6,
    ),
)

def to_V(MHz) : return constants.factor__VpMHz * MHz

In [4]:
wigner_time.adwin.output(adwin.add(init(),connections,devices))

UndefinedVariableError: name 'np' is not defined

In [3]:
def init() :
    """
    Creates an experimental timeline for the initialization of every device.
    """
    return tl.create(
        lockbox_MOT__V=0.0,
        coil_compensationX__A= constants.compensation_magnetic_field_X__A,
        coil_compensationY__A= constants.compensation_magnetic_field_Y__A,
        coil_MOTlowerPlus__A = -constants.compensation_magnetic_field__A,
        coil_MOTupperPlus__A = constants.compensation_magnetic_field__A,
        AOM_MOT = 1,
        AOM_repump = 1,
        AOM_OP_aux = 0,
        AOM_OP = 1,
        shutter_MOT = 0,
        shutter_repump = 0,
        shutter_OP001 = 0,
        shutter_OP002 = 1,
        context="ADwin_LowInit"
    )


class preparation_functions:
    def MOT(tline, time, lA, uA) :
        return tl.set(
            ["shutter_MOT", 1],
            ["shutter_repump", 1],
            ["coil_MOTlower__A", lA],
            ["coil_MOTupper__A", uA],
            t=time,timeline=tline,context='MOT',
         )
        
    def MOT_detunedGrowth(tline, time, duration, toMHz, pt = 3) :
        return tl.next(
            ["lockbox_MOT__V", lbox.to_V(toMHz)],
            duration=duration,
            fargs={"ti": pt},
            time_start=time,timeline=tline,context='MOT',
        )
    
    def molasses(tline, time, cooling_duration, coil_duration_ramp, lockbox_duration_ramp, toMHz, coil_parameter_tanh = 3, lockbox_parameter_tanh = 3) :
        """
        Creates an experimental timeline for making optical molasses.
        """
        context = "molasses"
        return tl.update(
            tl.set(["AOM_MOT", [time+cooling_duration, 0]],
                   ["shutter_MOT", [time + cooling_duration - constants.lag_MOTshutter, 0]],
                    # here, the switch off was duplicated @(time + cooling_duration)
                timeline=tline,context=context,
            ),
            tl.next(
                ["coil_MOTlower__A", 0],
                ["coil_MOTupper__A", 0], # TODO: can these be other than 0 (e.g. for more perfect compensaton?)
                time_start=time,
                duration=coil_duration_ramp,
                fargs={"ti": coil_parameter_tanh},
                context=context,
            ),
            tl.next(
                ["lockbox_MOT__V", lbox.to_V(toMHz)],
                time_start=time,
                duration=lockbox_duration_ramp,
                fargs={"ti": lockbox_parameter_tanh},
                context=context,
            ),
        )
    
    def OP(tline, time, duration_exposure, duration_ramp_coils_dipole, homogenous_field__A,pt=3) :
        """
        Creates an experimental timeline for optical pumping.
        """
        context = "OP"
        return tl.update(
            tl.set(
                [
                    "AOM_OP",
                    [ time - constants.OP.lag_aom_on - constants.OP.lag_shutter_on, 0 ],
                ],
                # I switch off the AOM close before opening the first shutter, but safely enough, not to flash too soon
                [
                    "shutter_OP001",
                    [ time - constants.OP.lag_aom_on - constants.safety_factor * (constants.OP.lag_shutter_on + constants.OP.duration_shutter_on), 1 ],
                ],
                [
                    "AOM_OP",
                    [time - constants.OP.lag_aom_on + 9.5e-6, 1], #TODO: what is this constant?????
                ],
                [
                    "shutter_OP002",
                    [ time + duration_exposure - (2 - constants.safety_factor) * constants.OP.lag_shutter_off, 0 ],
                ],
                ["AOM_OP", [ time+duration_exposure + 0.5e-6, 0 ] ], # TODO: another mysterious constant
                ["AOM_repump", [ time+duration_exposure, 0 ] ],
                ["shutter_repump", [ time+duration_exposure, 0 ] ],
                # Shutters switched back, to reinitialize them before any additional optical pumpings later.
                ["shutter_OP001", [ time+duration_exposure, 0 ] ],
                ["shutter_OP002", [ time+duration_exposure + constants.safety_factor * (constants.OP.lag_shutter_on+constants.OP.duration_shutter_on), 1 ] ],
                context=context,
                timeline=tline,
            ),
            tl.next(
                ["coil_MOTlower__A", homogenous_field__A],
                ["coil_MOTupper__A", -homogenous_field__A],
                time_start=time - duration_ramp_coils_dipole,
                fargs={"ti": pt},
                duration=duration_ramp_coils_dipole,
                context=context,
            ),
        )
    
    
    def pull_coils(tline, time, duration, l, u, lp = - constants.compensation_magnetic_field__A, up = constants.compensation_magnetic_field__A, pt = 3, context="") :
        return tl.update(
            tl.next(
                ["coil_MOTlower__A", l],
                ["coil_MOTupper__A", u],
                ["coil_MOTlowerPlus__A", lp],
                ["coil_MOTupperPlus__A", up],
                fargs={"ti": pt},
                time_start=time,duration=duration,timeline=tline,context=context,
            ),
        )
    

    def finish(tline, time, duration_ramp, lockbox_initial__V, science_AOM__V, coil_lowerMOT__A, coil_upperMOT__A,ending_parameter_tanh=3) :
        """
        The above but with the new version of next.
        """
        context = "finish"

        return tl.update(
            tl.set(
                timeline=tline,
                context="finish",
                t=duration_total,
                AOM_MOT=1,
                AOM_repump=1,
                AOM_repump__V=5,
                AOM_imaging=1,
                AOM_OP_aux=0,
                AOM_OP=1,
                AOM_science=1,
                AOM_science__V= science_AOM__V,
                AOM_ref=1,
                shutter_MOT=1,
                shutter_repump=1,
                shutter_imaging=0,
                shutter_OP001=0,
                shutter_OP002=1,
                shutter_science=1,
                shutter_transversePump=0,
                shutter_coupling=0,
                trigger_TC__V=0,
                photodiode__V=0,
            ),
            tl.next(
                ["coil_MOTlower__A", coil_lowerMOT__A],
                ["coil_MOTupper__A", coil_upperMOT__A],
                ["coil_MOTlowerPlus__A", -constants.compensation_magnetic_field__A],
                ["coil_MOTupperPlus__A", constants.compensation_magnetic_field__A],
                ["coil_compensationX__A", constants.compensation_magnetic_field_X__A],
                ["coil_compensationY__A", constants.compensation_magnetic_field_Y__A],
                ["lockbox_MOT__V", lbox.to_V(0.0)],
                # ["lockbox_science__V", params.experiment_CQED.science_light.lockbox_initial__V],
                # ["trigger_RF_modulation__V", 0],
                fargs={"ti": ending_parameter_tanh},
                duration=duration_safely_ramp,
                time_start=time_ramps_before_finish,
                context="before_finish",
            ),
        )

In [164]:
params_default = Munch(
    MOT=Munch(
        duration_grow=15.0,
        duration_final=100e-3, # should be smaller than duration_grow
        shift__MHz=-5.0,
        duration_ramp=10e-3,
        coil_lower__A=-1,
        coil_upper__A=-0.98, # TODO why the asymmetry? this was probably part of magnetic compensation
    ),
    molasses=Munch(
        duration_cooling=5e-3,
        duration_ramp=1e-3, # should be smaller than duration_cooling
        shift__MHz=-80,
        fraction_ramp_duration=0.9,
        ),
    OP=Munch(
        duration_exposure=80e-6,
        duration_ramp_coils_dipole=500e-6,
        homogeneous_field__A=-0.12,
    ),
    trapping_magnetic=Munch(
        delay_magnetic_trap=1e-3,
        duration_ramp_coils_quadrupole=50e-6,
        duration_ramp_coils_strengthen=3e-3,
        lowerMOT_quadrupole__A=-1.8,
        upperMOT_quadrupole__A=-1.7, # TODO why the asymmetry? this was probably part of magnetic compensation
        lowerMOT_strengthen__A=-4.8,
        upperMOT_strengthen__A=-4.7, # TODO why the asymmetry? this was probably part of magnetic compensation
    ),
    transport_magnetic=Munch(
        delay=5e-3,
        duration=150e-3,
        lowerMOT__A=-0.3121632,
        upperMOT__A=-4.748368,
        lowerMOTPlus__A=-4.648368,
        upperMOTPlus__A=-4.748368,
        parameter_tanh=2.25,
    ),
    finish=Munch(
        duration_until_finish = 0,
        duration_ramp = 1e-3,
        lockbox_initial__V = 0,
    )
)

In [186]:
def init():
    return preparation_functions.init()

def MOT(tline, time, params,context=""):
    tline = preparation_functions.MOT(tline, time, params.MOT.lowerMOT__A, params.MOT.upperMOT__A)
    tline = preparation_functions.MOT_detunedGrowth(tline, time+params.MOT.duration_grow-params.MOT.duration_final, params.MOT.duration_ramp, 
                                             params.MOT.shift__MHz,pt=3)
    return tline, time+params.MOT.duration_grow

def molasses(tline, time, params):
    tline = preparation_functions.molasses(tline, time, params.molasses.duration_cooling, params.molasses.duration_ramp, params.molasses.duration_ramp, 
                                            params.molasses.shift__MHz)
    return tline, time+params.molasses.duration_cooling

def OP(tline, time, params):
    #def OP(tline, time, duration_exposure, duration_ramp_coils_dipole, duration_shutter_on, homogenous_field__A,coil_parameter_tanh=3):
    tline = preparation_functions.OP(tline, time, params.OP.duration_exposure, params.OP.duration_ramp_coils_dipole, params.OP.homogenous_field__A)
    return tline, time+params.OP.duration_exposure

def magnetic_field(tline, time, params):
    tline = preparation_functions.pull_coils(tline, time, params.trapping_magnetic.duration_ramp_coils_quadrupole, params.trapping_magnetic.lowerMOT_quadrupole__A,
                                                    params.trapping_magnetic.upperMOT_quadrupole__A,context="trapping_magnetic")
    time = time+params.trapping_magnetic.duration_ramp_coils_quadrupole
    tline = preparation_functions.pull_coils(tline, time, params.trapping_magnetic.duration_ramp_coils_strengthen, params.trapping_magnetic.lowerMOT_strengthen__A,
                                             params.trapping_magnetic.upperMOT_strengthen__A,context="trapping_magnetic")
    return tline, time+params.trapping_magnetic.duration_ramp_coils_strengthen

def transport_magnetic(tline, time, params):
    tline = preparation_functions.pull_coils(tline, time, params.transport_magnetic.duration, params.transport_magnetic.lowerMOT__A,
                                            params.transport_magnetic.upperMOT__A,params.transport_magnetic.lowerMOTPlus__A,params.transport_magnetic.upperMOTPlus__A,
                                            context = "magnetic_transport")
    return tline, time+params.transport_magnetic.duration

def switch_off_trap_magnetic(tline, time, params):
    tline = preparation_functions.pull_coils(tline, time, params.switch_off_trap_magnetic.duration, 0,0)
    return tline, time+params.switch_off_trap_magnetic.duration

#parameter??
def transport_back_magnetic(tline, time, params):
    tline = preparation_functions.transport_back_magnetic(tline, time, params.transport_back_magnetic.duration, params.transport_back_magnetic.lowerMOT_strengthen__A, params.transport_back_magnetic.upperMOT_strengthen__A)
    return tline, time+params.transport_back_magnetic.duration

def finish(tline, time, params):
    return preparation_functions.finish(tline, time, params.finish.duration_ramp, params.finish.lockbox_initial__V, params.finish.science_AOM__V, params.finish.coil_lowerMOT__A, params.finish.coil_upperMOT__A)

In [190]:
def prepare_atoms(params):
    tline = init()
    tline,timeMOT = MOT(tline, 0, params)
    tline,timemolasses = molasses( *MOT(tline, 0, params), params)
    tline,timeOP = OP(tline, timemolasses, params)
    tline,timeTrapping = magnetic_field(tline, timeOP+params.trapping_magnetic.delay_magnetic_trap, params)
    tline = finish(tline, timeTrapping, params)
    return tline

In [191]:
params=deepcopy(params_default)
params.finish.duration_until_finish = 500e-3

tline = prepare_atoms(params)

In [192]:
%matplotlib qt
tl.display(tline,variables=['lockbox_MOT__V','AOM_MOT','AOM_repump','shutter_repump','AOM_repump__V'],xlim=[15,15.05])

0 Axes(0.125,0.747241;0.775x0.132759) lockbox_MOT__V
1 Axes(0.125,0.587931;0.775x0.132759) AOM_MOT
2 Axes(0.125,0.428621;0.775x0.132759) AOM_repump
3 Axes(0.125,0.26931;0.775x0.132759) shutter_repump
4 Axes(0.125,0.11;0.775x0.132759) AOM_repump__V
